In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [5]:
# Initialize Selenium WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [6]:
# Function to extract product links from a category page
def get_product_links(category_url, num_pages=25):
    product_links = []
    for page in range(1, num_pages + 1):
        driver.get(f"{category_url}?page={page}")
        time.sleep(5)  # Wait for the page to load
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Update the selector to match Konga's HTML structure for product links
        for link in soup.select('ul > li > div > div > a'):
            product_links.append('https://www.konga.com' + link['href'])
    
    return product_links

In [7]:
# Function to extract reviews from a product page
def get_reviews(product_url):
    driver.get(product_url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    reviews = []
    
    while True:
        # Update the selector to match Konga's HTML structure for reviews
        review_elements = soup.select('div._42878_20oAo > div:nth-child(4) > div')
        for element in review_elements:
            review_text = element.find('p').text.strip()
            rating = element.find('div').text.strip()  # Update based on actual HTML structure
            reviews.append({'product_url': product_url, 'review_text': review_text, 'rating': rating})

        # Check for the presence of a 'Next' button and click it if found
        next_button = soup.find('a', {'aria-label': 'Next'})
        if next_button:
            next_button.click()
            time.sleep(5)
        else:
            break
    
    return reviews

In [8]:
# Main function to scrape reviews from the Fashion category
def scrape_konga_fashion_reviews():
    category_url = 'https://www.konga.com/category/konga-fashion-1259'  # Replace with the actual category URL
    product_links = get_product_links(category_url, num_pages=25)  # Set to 25 to scrape all pages
    all_reviews = []

    for product_url in product_links:
        reviews = get_reviews(product_url)
        all_reviews.extend(reviews)

    # Save all reviews to a DataFrame
    df = pd.DataFrame(all_reviews)
    df.to_csv('konga_fashion_reviews.csv', index=False)

    # Close the driver
    driver.quit()

# Run the scraping process
scrape_konga_fashion_reviews()

NameError: name 'konga_scraper' is not defined